# CONSTANTS

In [1]:
FASTQC_PATH = "/home/analytics/distr/FastQC/fastqc"
TRIMMOMATIC_JAR = "/home/analytics/bin/trimmomatic.jar"
BOWTIE_DIR = "/home/analytics/distr/bowtie2-2.3.4.3-linux-x86_64/"
SAMTOOLS_DIR = "/home/analytics/distr/samtools-1.9/build/bin"
VARSCAN_DIR = "/home/analytics/distr/varscan/"

# Data fetching

In [2]:
!mkdir -p ../data/
!wget -P ../data/ ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Escherichia_coli/all_assembly_versions/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.fna.gz 2> /dev/null
!wget -P ../data/ ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Escherichia_coli/all_assembly_versions/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.gff.gz 2> /dev/null

In [3]:
!wget -P ../data/ http://public.dobzhanskycenter.ru/mrayko/amp_res_1.fastq.zip 2> /dev/null
!wget -P ../data/ http://public.dobzhanskycenter.ru/mrayko/amp_res_2.fastq.zip 2> /dev/null

In [4]:
!unzip -o ../data/amp_res_1.fastq.zip -d ../data/
!unzip -o ../data/amp_res_2.fastq.zip -d ../data/

Archive:  ../data/amp_res_1.fastq.zip
  inflating: ../data/amp_res_1.fastq  
Archive:  ../data/amp_res_2.fastq.zip
  inflating: ../data/home/mike/UCSD/BIMM185/Week1/amp_res_2.fastq  
  inflating: ../data/amp_res_2.fastq  


# FastQC analysis

In [5]:
!$FASTQC_PATH ../data/amp_res_1.fastq ../data/amp_res_2.fastq 2> /dev/null

Analysis complete for amp_res_1.fastq
Analysis complete for amp_res_2.fastq


I'd like to include htmls into cell outputs, so let's get a bucket on AWS and use boto for sending static htmls to the host.

In [6]:
from boto.s3 import connect_to_region
from boto import connect_s3
from boto.s3.key import Key

In [7]:
AWS_S3_ROOT = "http://bioinf-workshop.s3-website.eu-central-1.amazonaws.com/"

def perform_file_upload(file_, file_name):

    conn = connect_to_region("eu-central-1", 
                             aws_access_key_id=open("../.access_key", "r").read().strip(),
                             aws_secret_access_key=open("../.secret_key", "r").read().strip())

    bucket = conn.get_bucket("bioinf-workshop")
    key = Key(bucket)
    key.key = file_name
    key.set_contents_from_string(file_, headers={"Content-Type": "text/html"})
    key.set_acl('public-read')
    return AWS_S3_ROOT + file_name # public url to access the uploaded file


In [8]:
f_1 = perform_file_upload(open("../data/amp_res_1_fastqc.html", "r").read(), "week_1/fastqc_1.html")
f_2 = perform_file_upload(open("../data/amp_res_2_fastqc.html", "r").read(), "week_1/fastqc_2.html")

In [9]:
from IPython.display import IFrame

IFrame(src=f_1, width=1000, height=600)

In [10]:
from IPython.display import IFrame

IFrame(src=f_2, width=1000, height=600)

#### Due to presence of '#', ':' etc it's likely that this fastq file has Phred33 scale.

In [11]:
ord(":"), ord("#")

(58, 35)

There are several problems with raw reads:  
* Low quality at the end of reads
* Systematic error with forward reads (bubble or something like this)
* Severely non-uniform distribution of nucleotides at the start of the reads

We can adreess the problems using trimmomatic. After several tries I ended up with the following configuration:
* Do not crop adapters, probably these adapters are not usual
* Crop 20 nucleotides from the start instead
* Use sliding window so as to drop low quality at the end and tile problems
* Minimal length = 30 as a 'rule of thumb'

In [12]:
!java -jar $TRIMMOMATIC_JAR PE \
    -phred33 \
    -trimlog ../data/trim.log \
    ../data/amp_res_1.fastq ../data/amp_res_2.fastq \
    ../data/amp_res_1_p.fastq ../data/amp_res_1_u.fastq ../data/amp_res_2_p.fastq ../data/amp_res_2_u.fastq \
    HEADCROP:20 SLIDINGWINDOW:4:25 MINLEN:30

TrimmomaticPE: Started with arguments: -phred33 -trimlog ../data/trim.log ../data/amp_res_1.fastq ../data/amp_res_2.fastq ../data/amp_res_1_p.fastq ../data/amp_res_1_u.fastq ../data/amp_res_2_p.fastq ../data/amp_res_2_u.fastq HEADCROP:20 SLIDINGWINDOW:4:25 MINLEN:30
Multiple cores found: Using 16 threads
Input Read Pairs: 455876 Both Surviving: 337781 (74.09%) Forward Only Surviving: 42881 (9.41%) Reverse Only Surviving: 30468 (6.68%) Dropped: 44746 (9.82%)
TrimmomaticPE: Completed successfully


In [13]:
!$FASTQC_PATH ../data/amp_res_1_p.fastq ../data/amp_res_2_p.fastq 2> /dev/null

Analysis complete for amp_res_1_p.fastq
Analysis complete for amp_res_2_p.fastq


In [14]:
f_1 = perform_file_upload(open("../data/amp_res_1_p_fastqc.html", "r").read(), "week_1/fastqc_p_1.html")
f_2 = perform_file_upload(open("../data/amp_res_2_p_fastqc.html", "r").read(), "week_1/fastqc_p_2.html")

In [15]:
from IPython.display import IFrame

IFrame(src=f_1, width=1000, height=600)

In [16]:
from IPython.display import IFrame

IFrame(src=f_2, width=1000, height=600)

Now reads look suitable for further analysis.

# Build index

In [17]:
!$BOWTIE_DIR/bowtie2-build -h

Bowtie 2 version 2.3.4.3 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: bowtie2-build [options]* <reference_in> <bt2_index_base>
    reference_in            comma-separated list of files with ref sequences
    bt2_index_base          write bt2 data to files with this dir/basename
*** Bowtie 2 indexes work only with v2 (not v1).  Likewise for v1 indexes. ***
Options:
    -f                      reference files are Fasta (default)
    -c                      reference sequences given on cmd line (as
                            <reference_in>)
    --large-index           force generated index to be 'large', even if ref
                            has fewer than 4 billion nucleotides
    --debug                 use the debug binary; slower, assertions enabled
    --sanitized             use sanitized binary; slower, uses ASan and/or UBSan
    --verbose               log the issued command
    -a/--noauto             disable automatic -p/--bmax/--dcv memory-fitting
   

In [18]:
!$BOWTIE_DIR/bowtie2-build ../data/GCA_000005845.2_ASM584v2_genomic.fna.gz ../data/E-coli.idx

Settings:
  Output files: "../data/E-coli.idx.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ../data/GCA_000005845.2_ASM584v2_genomic.fna.gz
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1160413
Using parameters --bmax 870310 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 87031

# Align reads

In [19]:
!$BOWTIE_DIR/bowtie2 -x ../data/E-coli.idx \
    -1 ../data/amp_res_1_p.fastq \
    -2 ../data/amp_res_2_p.fastq \
    -p 20 > ../data/alignment.sam

337781 reads; of these:
  337781 (100.00%) were paired; of these:
    42283 (12.52%) aligned concordantly 0 times
    287309 (85.06%) aligned concordantly exactly 1 time
    8189 (2.42%) aligned concordantly >1 times
    ----
    42283 pairs aligned concordantly 0 times; of these:
      29947 (70.83%) aligned discordantly 1 time
    ----
    12336 pairs aligned 0 times concordantly or discordantly; of these:
      24672 mates make up the pairs; of these:
        20568 (83.37%) aligned 0 times
        2385 (9.67%) aligned exactly 1 time
        1719 (6.97%) aligned >1 times
96.96% overall alignment rate


In [20]:
!head -n 10 ../data/alignment.sam

@HD	VN:1.0	SO:unsorted
@SQ	SN:U00096.3	LN:4641652
@PG	ID:bowtie2	PN:bowtie2	VN:2.3.4.3	CL:"/home/analytics/distr/bowtie2-2.3.4.3-linux-x86_64/bowtie2-align-s --wrapper basic-0 -x ../data/E-coli.idx -p 20 -1 ../data/amp_res_1_p.fastq -2 ../data/amp_res_2_p.fastq"
SRR1363257.37	99	U00096.3	232669	42	43M	=	232779	150	GCTGTTCCAGCGCATCACATCTTTGATGTTCACGCCGTGGCGT	@E:FFHGAE4?C?DE<BFGEC>?>FHE4BFFIIFHIBABEECA	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:43	YS:i:0	YT:Z:CP
SRR1363257.37	147	U00096.3	232779	42	40M	=	232669	-150	GTCGCTGTGCGCTACTGCCTGCACCAATCGTCAAACTTTG	DHHE9AA8F<<8D?F;@>D@??:B:9@EGHHFHBA;HGFA	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:40	YS:i:0	YT:Z:CP
SRR1363257.78	99	U00096.3	2481311	42	80M	=	2481359	129	GAGTAAAATAAATACGCGCATGTGATACTCACAATACCAATGGTGAAGTTACGGGACTTAAACAAACTGAGATCAAGAAT	JJJFFHIJJJJJJJJJJJJJJJJJJJJJJJIJHHHHHHFDEDF;AEEEEEEDDDDDBBACDDDCDDDDCCDDDDDDCCDC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:80	YS:i:0	YT:Z:CP
SRR1363257.78	147	U00096.3	2481359	42	81M	=	248

In [21]:
!$SAMTOOLS_DIR/samtools view -S -b ../data/alignment.sam > ../data/alignment.bam

In [22]:
!$SAMTOOLS_DIR/samtools flagstat ../data/alignment.bam

675562 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
654994 + 0 mapped (96.96% : N/A)
675562 + 0 paired in sequencing
337781 + 0 read1
337781 + 0 read2
590996 + 0 properly paired (87.48% : N/A)
652610 + 0 with itself and mate mapped
2384 + 0 singletons (0.35% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


88.6% of read pairs has been successfully mapped. It looks 'okay'

In [23]:
!$SAMTOOLS_DIR/samtools sort \
    ../data/alignment.bam \
    -o ../data/alignment_sorted.bam

In [24]:
!$SAMTOOLS_DIR/samtools index ../data/alignment_sorted.bam

# Variant calling

In [25]:
!gunzip ../data/GCA_000005845.2_ASM584v2_genomic.fna.gz

In [26]:
!$SAMTOOLS_DIR/samtools mpileup \
    -f ../data/GCA_000005845.2_ASM584v2_genomic.fna \
    ../data/alignment_sorted.bam >  ../data/Ecoli.mpileup

[mpileup] 1 samples in 1 input files


In [27]:
!head -n 10 ../data/Ecoli.mpileup

U00096.3	1	A	1	^K.	E
U00096.3	2	G	1	.	J
U00096.3	3	C	1	.	J
U00096.3	4	T	1	.	J
U00096.3	5	T	2	..	JJ
U00096.3	6	T	2	..	JJ
U00096.3	7	T	3	..^K.	JJE
U00096.3	8	C	3	...	IJJ
U00096.3	9	A	3	...	GJJ
U00096.3	10	T	3	...	GJJ


In [28]:
!java -jar $VARSCAN_DIR/VarScan.v2.4.0.jar mpileup2snp -h

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.2
Min avg qual:	15
P-value thresh:	0.01
USAGE: java -jar VarScan.jar mpileup2cns [pileup file] OPTIONS
	mpileup file - The SAMtools mpileup file

	OPTIONS:
	--min-coverage	Minimum read depth at a position to make a call [8]
	--min-reads2	Minimum supporting reads at a position to call variants [2]
	--min-avg-qual	Minimum base quality at a position to count a read [15]
	--min-var-freq	Minimum variant allele frequency threshold [0.01]
	--min-freq-for-hom	Minimum frequency to call homozygote [0.75]
	--p-value	Default p-value threshold for calling variants [99e-02]
	--strand-filter	Ignore variants with >90% support on one strand [1]
	--output-vcf	If set to 1, outputs in VCF format
	--vcf-sample-list	For VCF output, a list of sample names in order, one per line
	--variants	Report only variant (SNP/indel) positions [0]


In [29]:
!java -jar $VARSCAN_DIR/VarScan.v2.4.0.jar \
    mpileup2snp ../data/Ecoli.mpileup --min-var-freq .05 \
    --variants \
    --output-vcf 1 > ../data/VarScan_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.05
Min avg qual:	15
P-value thresh:	0.01
Reading input from ../data/Ecoli.mpileup
4634226 bases in pileup file
5 variant positions (5 SNP, 0 indel)
0 were failed by the strand-filter
5 variant positions reported (5 SNP, 0 indel)


In [30]:
!head -n 40 ../data/VarScan_results.vcf

##fileformat=VCFv4.1
##source=VarScan2
##INFO=<ID=ADP,Number=1,Type=Integer,Description="Average per-sample depth of bases with Phred score >= 15">
##INFO=<ID=WT,Number=1,Type=Integer,Description="Number of samples called reference (wild-type)">
##INFO=<ID=HET,Number=1,Type=Integer,Description="Number of samples called heterozygous-variant">
##INFO=<ID=HOM,Number=1,Type=Integer,Description="Number of samples called homozygous-variant">
##INFO=<ID=NC,Number=1,Type=Integer,Description="Number of samples not called">
##FILTER=<ID=str10,Description="Less than 10% or more than 90% of variant supporting reads on one strand">
##FILTER=<ID=indelError,Description="Likely artifact due to indel reads at this position">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=SDP,Number=1,Type=Integer,Description="Raw Read Depth as reported by SAMtools">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Quality

In [31]:
!gunzip ../data/GCA_000005845.2_ASM584v2_genomic.gff.gz

In [32]:
!cat ../data/GCA_000005845.2_ASM584v2_genomic.gff | awk '{ if ($4 <= 482698 && $5 >= 482698) { print $0 } }'

U00096.3	Genbank	region	1	4641652	.	+	.	ID=id0;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655
U00096.3	Genbank	gene	481254	484403	.	-	.	ID=gene477;Dbxref=ASAP:ABE-0001601,ECOCYC:EG11704,EcoGene:EG11704;Name=acrB;gbkey=Gene;gene=acrB;gene_biotype=protein_coding;gene_synonym=acrE,ECK0456;locus_tag=b0462
U00096.3	Genbank	CDS	481254	484403	.	-	0	ID=cds459;Parent=gene477;Dbxref=UniProtKB/Swiss-Prot:P31224,NCBI_GP:AAC73564.1,ASAP:ABE-0001601,ECOCYC:EG11704,EcoGene:EG11704;Name=AAC73564.1;gbkey=CDS;gene=acrB;orig_transcript_id=gnl|b0462|mrna.b0462;product=multidrug efflux pump RND permease AcrB;protein_id=AAC73564.1;transl_table=11


In [33]:
fasta_lines = open("../data/GCA_000005845.2_ASM584v2_genomic.fna", "r").readlines()
genome_string = "".join([l.strip() for l in fasta_lines[1:]])

In [34]:
from IPython.display import IFrame

IFrame(src="https://en.wikipedia.org/wiki/DNA_codon_table", width=1000, height=600)

In [35]:
codone_pos = (482698 - 481254) % 3
codone_start = 482698 - codone_pos - 1
codone = genome_string[codone_start:codone_start+3]
print(codone, codone[:codone_pos] + "A" + codone[codone_pos+1:])

CTG CAG


CTG -> CAG means Leucine to Glutamine substitution. It is a missense mutation that potentially could explain differences in response to antibiotics.

In [36]:
!cat ../data/GCA_000005845.2_ASM584v2_genomic.gff | awk '{ if ($4 <= 852762 && $5 >= 852762) { print $0 } }'

U00096.3	Genbank	region	1	4641652	.	+	.	ID=id0;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655
U00096.3	Genbank	gene	852725	853064	.	-	.	ID=gene842;Dbxref=ASAP:ABE-0047240,ECOCYC:G0-8881;Name=rybA;gbkey=Gene;gene=rybA;gene_biotype=ncRNA;gene_synonym=ECK0806;locus_tag=b4416
U00096.3	Genbank	ncRNA	852725	853064	.	-	.	ID=rna30;Parent=gene842;Dbxref=ASAP:ABE-0047240,ECOCYC:G0-8881;gbkey=ncRNA;gene=rybA;product=small RNA RybA
U00096.3	Genbank	exon	852725	853064	.	-	.	ID=id228;Parent=rna30;Dbxref=ASAP:ABE-0047240,ECOCYC:G0-8881;gbkey=ncRNA;gene=rybA;product=small RNA RybA


In [37]:
codone_pos = (852762 - 852725) % 3
codone_start = 852762 - codone_pos - 1
codone = genome_string[codone_start:codone_start+3]
print(codone, codone[:codone_pos] + "G" + codone[codone_pos+1:])

AAA AGA


AAA -> AGA means Lysine to Arginine substitution. It is a conservative substitution.

In [38]:
!cat ../data/GCA_000005845.2_ASM584v2_genomic.gff | awk '{ if ($4 <= 1905761 && $5 >= 1905761) { print $0 } }'

U00096.3	Genbank	region	1	4641652	.	+	.	ID=id0;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655
U00096.3	Genbank	gene	1905688	1906254	.	+	.	ID=gene1904;Dbxref=ASAP:ABE-0006065,ECOCYC:G6999,EcoGene:EG14012;Name=mntP;gbkey=Gene;gene=mntP;gene_biotype=protein_coding;gene_synonym=ECK1819,yebN;locus_tag=b1821
U00096.3	Genbank	CDS	1905688	1906254	.	+	0	ID=cds1844;Parent=gene1904;Dbxref=UniProtKB/Swiss-Prot:P76264,NCBI_GP:AAC74891.2,ASAP:ABE-0006065,ECOCYC:G6999,EcoGene:EG14012;Name=AAC74891.2;gbkey=CDS;gene=mntP;orig_transcript_id=gnl|b1821|mrna.b1821;product=Mn(2(+)) exporter;protein_id=AAC74891.2;transl_table=11


In [39]:
codone_pos = (1905761 - 1905688) % 3
codone_start = 1905761 - codone_pos - 1
codone = genome_string[codone_start:codone_start+3]
print(codone, codone[:codone_pos] + "A" + codone[codone_pos+1:])

GGT GAT


GGT -> GAT means Glycine to Aspartate substitution. It is a missense mutation that potentially could explain differences in response to antibiotics.

In [40]:
!cat ../data/GCA_000005845.2_ASM584v2_genomic.gff | awk '{ if ($4 <= 4296060 && $5 >= 4296060) { print $0 } }'

U00096.3	Genbank	region	1	4641652	.	+	.	ID=id0;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655


Here is just a substitution in a 'middle of nowhere' region. In fact, it could explain resistance too, but through different mechanisms, like regulation of gene expression. However evidence we've got are barely enough to say something about that.

In [41]:
!cat ../data/GCA_000005845.2_ASM584v2_genomic.gff | awk '{ if ($4 <= 4390754 && $5 >= 4390754) { print $0 } }'

U00096.3	Genbank	region	1	4641652	.	+	.	ID=id0;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655
U00096.3	Genbank	gene	4390457	4391509	.	-	.	ID=gene4314;Dbxref=ASAP:ABE-0013626,ECOCYC:G7841,EcoGene:EG12479;Name=rsgA;gbkey=Gene;gene=rsgA;gene_biotype=protein_coding;gene_synonym=ECK4157,yjeQ;locus_tag=b4161
U00096.3	Genbank	CDS	4390457	4391509	.	-	0	ID=cds4118;Parent=gene4314;Dbxref=UniProtKB/Swiss-Prot:P39286,NCBI_GP:AAC77121.2,ASAP:ABE-0013626,ECOCYC:G7841,EcoGene:EG12479;Name=AAC77121.2;gbkey=CDS;gene=rsgA;orig_transcript_id=gnl|b4161|mrna.b4161;product=ribosome small subunit-dependent GTPase A;protein_id=AAC77121.2;transl_table=11


In [42]:
codone_pos = (4390754 - 4390457) % 3
codone_start = 4390754 - codone_pos - 1
codone = genome_string[codone_start:codone_start+3]
print(codone, codone[:codone_pos] + "T" + codone[codone_pos+1:])

GGC TGC


GGC -> TGC means Glycine to Cysteine substitution. It is a missense mutation that potentially could explain differences in response to antibiotics.